# Food Safety and Community Food Production

An exploration of how narratives of "safety" in land use policy impact urban agriculture in select CA Counties: LA, Ventura, Sonoma, Mendocino, and Lake Counties

## The Data

112 total general plans from the five CA Counties (LA, Ventura, Sonoma, Mendocino, Lake)


1. Basic word counts:

2. Modified/precision topic modeling: How often do mentions of food, agriculture, soil health, food safety feature in plans? And as important, where are they mentioned?

In [1]:
# libraries
import pandas as pd
import geopandas as gpd
import numpy as np

# visualizations
import contextily as ctx
import matplotlib.pyplot as plt
import seaborn as sn

# census
import cenpy
from cenpy import products

# set display
pd.options.display.max_columns = 150
#pd.options.display.max_rows = 150

In [2]:
# libraries
import os

# extract text
from pdfminer.high_level import extract_text
import re

# stop words; split sentences; stems
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer

# topic modeling
import gensim
help(gensim.models.LdaMulticore)
import pyLDAvis
import pyLDAvis.gensim_models

Help on class LdaMulticore in module gensim.models.ldamulticore:

class LdaMulticore(gensim.models.ldamodel.LdaModel)
 |  LdaMulticore(corpus=None, num_topics=100, id2word=None, workers=None, chunksize=2000, passes=1, batch=False, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001, random_state=None, minimum_probability=0.01, minimum_phi_value=0.01, per_word_topics=False, dtype=<class 'numpy.float32'>)
 |  
 |  An optimized implementation of the LDA algorithm, able to harness the power of multicore CPUs.
 |  Follows the similar API as the parent class :class:`~gensim.models.ldamodel.LdaModel`.
 |  
 |  Method resolution order:
 |      LdaMulticore
 |      gensim.models.ldamodel.LdaModel
 |      gensim.interfaces.TransformationABC
 |      gensim.utils.SaveLoad
 |      gensim.models.basemodel.BaseTopicModel
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, corpus=None, num_topics=100, id2word=None, workers=

#### The Plan Elements

After downloading all plans, the plans were split by element and only the "Land Use" and "Open Space/Conservation" Elements will be included.

In [5]:
# loading plans
planlist = os.listdir('C:/Users/melod/Documents/data science/Food-Systems-Policy-Research/Food Systems and General Plans/General Plans')
planlist

['.ipynb_checkpoints',
 'Alhambra General Plan.August 2019.lowres (PDF).pdf',
 'Cerritos.pdf',
 'City of Commerce 2020 General Plan.PDF',
 'Vernon_General_Plan.pdf']

In [6]:
# extracting text

# function
def readPDF(planname):
    txt = extract_text('C:/Users/melod/Documents/data science/Food-Systems-Policy-Research/Food Systems and General Plans/General Plans'+planname)
    
    # remove punctuation, numbers, etc.
    txt = re.sub(r"[^A-z\s]", "", txt)
    # remove whitepace
    txt = re.sub(r"\s+", " ", txt) 
    
    # cleaning up name
    planname = planname.split(".")[0]
    
    # insert muni(planname) to the beginning of plan: creating indexable muni ID within each plan string
    txt = planname+", "+txt
    
    print('Finished {}'.format(planname))
    return txt

# read in all pdf files
suffixes = ('.pdf', '.PDF')
genplan = [readPDF(pn) for pn in planlist if pn.endswith(suffixes)]

Finished Alhambra General Plan
Finished Cerritos
Finished City of Commerce 2020 General Plan
Finished Vernon_General_Plan


In [7]:
# establish list of stopwords to exclude
swords = stopwords.words('english')

#### Keyword Count (RAW TEXT VERSION: Space positions)

Loop to return only count of select keywords for each plan in genplan and store in stacked df.

In [16]:
# saving positions of keywords in raw plan text to a dictionary: Alhambra
textdict = {}

keywords = ['food', 'agriculture', 'garden', 'farm', 'fruit', 'vegetable', 'animal', 'soil', 
            'remediation', 'contaminate', 'carbon', 'sustainability', 'climate', 'environment', 
            'health', 'safety', 'justice']

for key in keywords:
    # text positions
    textpositions = [i for i in range(len(Alhambra)) if Alhambra.startswith(key, i)] # adapted: https://www.geeksforgeeks.org/python-all-occurrences-of-substring-in-string/#
    textdict[key] = textpositions
    
textdict

{'food': [28264,
  92775,
  102247,
  107926,
  109065,
  113179,
  178165,
  198174,
  198866,
  209796,
  209884,
  210207],
 'agriculture': [144819],
 'garden': [37125,
  39045,
  65339,
  97617,
  97662,
  97853,
  98611,
  102899,
  116871,
  118153,
  166246,
  166449,
  210144],
 'farm': [11630],
 'fruit': [101651],
 'vegetable': [98417, 101658, 210134],
 'animal': [124235],
 'soil': [18274,
  27578,
  121375,
  125060,
  178858,
  178959,
  178977,
  179005,
  179188,
  180326,
  180400,
  180684,
  180861,
  181002,
  182443,
  186816,
  203699,
  203796],
 'remediation': [],
 'contaminate': [],
 'carbon': [134877, 135016, 137259],
 'sustainability': [140202],
 'climate': [19215,
  19282,
  25232,
  25435,
  28188,
  33765,
  121436,
  121768,
  125225,
  125404,
  134458,
  134596,
  134781,
  137219,
  137295,
  137388,
  138817,
  138949,
  139884,
  149838,
  151106,
  152013,
  152585,
  199507,
  199858,
  200014,
  200077,
  200141,
  200369,
  201329,
  201949,
  20197

In [51]:
# converting dictionary into table counts: Alhambra
textdict2 = {}

for keyword in textdict:
    for position in keyword:
        textdict2[keyword] = len(textdict[keyword])

textdict2

{'food': 12,
 'agriculture': 1,
 'garden': 13,
 'farm': 1,
 'fruit': 1,
 'vegetable': 3,
 'animal': 1,
 'soil': 18,
 'remediation': 0,
 'contaminate': 0,
 'carbon': 3,
 'sustainability': 1,
 'climate': 39,
 'environment': 73,
 'health': 60,
 'safety': 32,
 'justice': 11}

In [12]:
# saving positions of keywords in raw plan text to a dictionary: all plans
textdictall = {}

keywords = ['food', 'agriculture', 'garden', 'farm', 'fruit', 'vegetable', 'animal', 'soil', 
            'remediation', 'contaminate', 'carbon', 'sustainability', 'climate', 'environment', 
            'health', 'safety', 'justice']

for plan in genplan:
    for key in keywords:
        # text positions:
        textpositionsall = [i for i in range(len(plan)) if plan.startswith(key, i)] # adapted: https://www.geeksforgeeks.org/python-all-occurrences-of-substring-in-string/#
        textdictall[key, plan.split(", ")[0]] = textpositionsall

textdictall

{('food', 'Alhambra General Plan'): [28264,
  92775,
  102247,
  107926,
  109065,
  113179,
  178165,
  198174,
  198866,
  209796,
  209884,
  210207],
 ('agriculture', 'Alhambra General Plan'): [144819],
 ('garden', 'Alhambra General Plan'): [37125,
  39045,
  65339,
  97617,
  97662,
  97853,
  98611,
  102899,
  116871,
  118153,
  166246,
  166449,
  210144],
 ('farm', 'Alhambra General Plan'): [11630],
 ('fruit', 'Alhambra General Plan'): [101651],
 ('vegetable', 'Alhambra General Plan'): [98417, 101658, 210134],
 ('animal', 'Alhambra General Plan'): [124235],
 ('soil', 'Alhambra General Plan'): [18274,
  27578,
  121375,
  125060,
  178858,
  178959,
  178977,
  179005,
  179188,
  180326,
  180400,
  180684,
  180861,
  181002,
  182443,
  186816,
  203699,
  203796],
 ('remediation', 'Alhambra General Plan'): [],
 ('contaminate', 'Alhambra General Plan'): [],
 ('carbon', 'Alhambra General Plan'): [134877, 135016, 137259],
 ('sustainability', 'Alhambra General Plan'): [140202]

In [13]:
# converting dictionary into table counts: all plans

plankeywordstxt = []

textdict3 = {}

for keyword in textdictall:
    for position in keyword:
        textdict3[keyword] = len(textdictall[keyword])

# turn dicts into dfs
    keywordscount = pd.DataFrame.from_dict(textdict3, orient='index', columns=['counts'])
    keywordscount.index.name = 'keyword'

# creating muni ID column
keywordscount['municipality'] = keywordscount.index
keywordscount['municipality'] = keywordscount['municipality'].astype('str')
keywordscount['municipality'] = keywordscount['municipality'].str.replace('(','').str.replace(')','')
keywordscount[['keyword', 'municipality']] = keywordscount['municipality'].str.split(',', expand = True)
keywordscount['municipality'] = keywordscount['municipality'].str.replace("' ",'').str.replace("'",'')
keywordscount['keyword'] = keywordscount['keyword'].str.replace("' ",'').str.replace("'",'')

# inspect
keywordscount

C:\Users\Urna6\AppData\Local\Temp\ipykernel_15316\29557323.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  keywordscount['municipality'] = keywordscount['municipality'].str.replace('(','').str.replace(')','')


,counts,municipality,keyword
keyword,,,
"(food, Alhambra General Plan)",12,Alhambra General Plan,food
"(agriculture, Alhambra General Plan)",1,Alhambra General Plan,agriculture
"(garden, Alhambra General Plan)",13,Alhambra General Plan,garden
"(farm, Alhambra General Plan)",1,Alhambra General Plan,farm
"(fruit, Alhambra General Plan)",1,Alhambra General Plan,fruit
...,...,...,...
"(climate, Vernon_General_Plan)",1,Vernon_General_Plan,climate
"(environment, Vernon_General_Plan)",43,Vernon_General_Plan,environment
"(health, Vernon_General_Plan)",21,Vernon_General_Plan,health


Visualizing mentions.

In [79]:
# uploading file with municipalities matched to Counties and assigning counties to municipalities in keyword df through join

# loading file
#counties = pd.read_csv('')

# resetting index for keyword df
#keywordcounts1 = keywordcounts.set_index('municipality')

# joining
#keywordlocassgnd = keywordcounts1.join(counties, how= 'left', rsuffix = '_counties')

#keywordscounts = 

In [82]:
# individual barplots (color-coded by north or south)

# municipalities = []

#fig ax, = plt.subplots(size= (8, 12))

#keywordscount.plot (ax=ax, column = 'keyword', size = 'counts', legend = True, legendkwds = {orientation: ''})

In [81]:
# aggregated: by keyword clusters: frequency north vs. south


### (B) Proximity of Keywords in Plans: How is Food and Urban Agriculture Being Talked About?

This next exercise aims to make inferences about how topics like food are being talked about based on the proximity (distance of words) between keywords (e.g. "food" and "soil" or "food" and "safety"). 

For each plan, keyword set pairs will be selected and matrix math will be used to compare the distance between the individual words in those keyword set pairs. The goal is to produce 28 matrices for word pairs for each plan for the following:

    Set 1: FOOD, AGRICULTURE, GARDEN, FARM
    Set 2: safety, soil, sustainability, climate, environment, remediation, justice, public, residential
    
-- and then to identify the frequency of word pairs that are within 100 or less words of each other in each municipality/plan. This would need to be done with a cleaned wordlist instead of the raw text, which might produce approximate word count distances for unique word pairs that exclude a lot of (filler) words, but would ultimately be more meaningful than distance measured by spaces in the raw text.

The hope is that this method will allow me to identify where (which municipalities) are talking about food and community agriculture in conjunction with environmental health and safety more generally.

In [21]:
# creating Alhambra plan object
Alhambra = genplan[0]

In [24]:
# dictionary of clean plan wordlist positions for keywords: Alhambra, keyset2

keywords1 = ['food', 'garden', 'farm', 'agriculture']

list1 = [word for word in word_tokenize(Alhambra.lower()) 
                 if word not in swords]

# create empty dictionary to store topic mention counts
lposdict1 = {}

for key1 in keywords1:
    # generate topic counts to populate dictionary frame indexed to topic
    positions = [i for i,w in enumerate(list1) if w.lower() == key1]
    lposdict1[key1] = positions

lposdict1

{'food': [9459, 9605, 10460, 11031, 11132, 11562, 18122, 20098, 20172, 21304],
 'garden': [4045, 9973, 9979, 9987, 10000, 10051, 10525, 12691, 14314],
 'farm': [1289],
 'agriculture': [14791]}

In [25]:
# dictionary of clean plan wordlist positions for keywords: Alhambra, keyset2

keywords2 = ['soil', 'remediation', 'sustainability', 'climate', 'environment', 'safety', 'justice', 
             'public', 'residential']

list1 = [word for word in word_tokenize(Alhambra.lower()) 
                 if word not in swords]

# create empty dictionary to store topic mention counts
lposdict2 = {}

for key2 in keywords2:
    # generate topic counts to populate dictionary frame indexed to topic
    positions = [i for i,w in enumerate(list1) if w.lower() == key2]
    lposdict2[key2] = positions

lposdict2

{'soil': [189,
  2841,
  11547,
  12421,
  12754,
  12803,
  18324,
  18333,
  18362,
  18378,
  18391,
  18959,
  20607,
  20622,
  20632,
  20643],
 'remediation': [9617],
 'sustainability': [2017, 14360],
 'climate': [224,
  2013,
  2019,
  2612,
  2634,
  2909,
  12462,
  12819,
  12839,
  13045,
  13120,
  13793,
  13805,
  13822,
  13837,
  14065,
  14071,
  14081,
  14222,
  14236,
  14331,
  15284,
  15413,
  15509,
  15570,
  20216,
  20219,
  20269,
  20281,
  20286,
  20293,
  20311,
  20407,
  20461,
  20465,
  20620,
  21148,
  21169,
  21190,
  21200,
  21213,
  21217,
  21230],
 'environment': [939,
  1227,
  1518,
  2158,
  2161,
  2284,
  2662,
  2732,
  2753,
  4334,
  4345,
  4821,
  4856,
  4938,
  5677,
  5708,
  6447,
  6911,
  8780,
  12978,
  14399,
  14418,
  14509,
  17428,
  17657,
  18732,
  18826,
  18846,
  18886,
  19767,
  19820,
  20127,
  20591],
 'safety': [210,
  952,
  1231,
  1507,
  1903,
  1918,
  1985,
  1987,
  2053,
  2659,
  2667,
  7785,
  8

Turning wordlists into arrays in order to do matrix math.

In [397]:
# (B) turn each keyset list in Alhambra dict into separate array that can be indexed/called in a loop

keywords1 = ['food', 'agriculture', 'garden', 'farm'] 
keywords2 = ['soil', 'remediation', 'sustainability', 'climate', 'environment', 'safety', 'justice']

keylist1store = []
keylist2store = []

for keyword, positions in lposdict.items():
    # turn position list in each keyword in keyset1 into an array
    for key1 in keywords1:
        key_list1 = []
        if key1 in keyword:
            key_list1 = positions # need to save all keywords to key_list1
            key_array1 = np.array(key_list1)
            keylist1store.append(key_array1)
            #keylist1store = np.array(keylist1store)

        # turn position list in each keyword in keyset2 into an array
        for key2 in keywords2:
            key_list2 = []
            if key2 in keyword:
                key_list2 = positions
                key_array2 = np.array(key_list2)
                keylist2store.append(key_array2)
              #  keylist2store = np.array(keylist2store)
        
            for array1 in keylist1store:
                for array2 in keylist2store:
                        # create a matrix from each unique keyset array pair
                    key1_key2 = abs(np.subtract(array1, array2[:,None])) # with different array shapes

                            # identify the unique keyword pair
                    print('"{}" and "{}"'.format(key1.upper(), key2.upper()))

                            # identify the number of occurences in the keyword pair where distance between them <100 words
                    close = np.count_nonzero(key1_key2 < 100)

                            # provide summary: ratio of close mentions to total mentions of each word
                    print('Mentions of "{}": {}'.format(key1, len(array1)))
                    print('Mentions of "{}": {}'.format(key2, len(key_array2)))
                    print('Mentions of "{}" within 100 words of "{}": {}'.format(key1, key2, close))

"FOOD" and "SOIL"
Mentions of "food": 10
Mentions of "soil": 16
Mentions of "food" within 100 words of "soil": 1
"FOOD" and "SOIL"
Mentions of "food": 1
Mentions of "soil": 16
Mentions of "food" within 100 words of "soil": 0
"FOOD" and "SOIL"
Mentions of "food": 9
Mentions of "soil": 16
Mentions of "food" within 100 words of "soil": 1
"FOOD" and "SOIL"
Mentions of "food": 1
Mentions of "soil": 16
Mentions of "food" within 100 words of "soil": 0
"FOOD" and "REMEDIATION"
Mentions of "food": 10
Mentions of "remediation": 16
Mentions of "food" within 100 words of "remediation": 1
"FOOD" and "REMEDIATION"
Mentions of "food": 1
Mentions of "remediation": 16
Mentions of "food" within 100 words of "remediation": 0
"FOOD" and "REMEDIATION"
Mentions of "food": 9
Mentions of "remediation": 16
Mentions of "food" within 100 words of "remediation": 1
"FOOD" and "REMEDIATION"
Mentions of "food": 1
Mentions of "remediation": 16
Mentions of "food" within 100 words of "remediation": 0
"FOOD" and "SUSTAI

In [27]:
keywords1 = ['food', 'garden', 'farm', 'agriculture']  
keywords2 = ['soil', 'remediation', 'sustainability', 'climate', 'environment', 'safety', 'justice',
            'public', 'residential']

proxim = {}

for key1, positions1 in lposdict1.items():
    for key2, positions2 in lposdict2.items():
        
        # create subtraction matrix for each keyword pair
        key1_key2 = abs(np.subtract(np.array(positions1), np.array(positions2)[:,None])) # with different array shapes
        # count occurences in each matrix where distance between keywords is <100 words + store in dictionary 
        proxim[key1, key2] = np.count_nonzero(key1_key2 < 100)
        # convert dictionary to dataframe
        prox = pd.DataFrame.from_dict(proxim, orient='index', columns=['Alhambra'])

prox

,Alhambra
"(food, soil)",1
"(food, remediation)",1
"(food, sustainability)",0
"(food, climate)",6
"(food, environment)",2
"(food, safety)",12
"(food, justice)",13
"(food, public)",15
"(food, residential)",0
"(garden, soil)",1


In [27]:
# for each plan
keywords1 = ['food', 'garden', 'farm', 'agriculture']  
keywords2 = ['soil', 'remediation', 'sustainability', 'climate', 'environment', 'safety', 'justice',
            'public', 'residential']

proxim = {}
plan_col = []

for plan in genplan:
    
    # create a wordlist for each plan
    wordlist = [word for word in word_tokenize(plan.lower()) 
                 if word not in swords]
    
    # create dictionaries for keyword set 1 for each plan
    lpostdict1 = {}
    for key1 in keywords1:
        # generate topic counts to populate dictionary frame indexed to topic
        positions1 = [i for i,w in enumerate(wordlist) if w.lower() == key1]
        lposdict1[key1] = positions1

    # create dictionaries for keyword set 2 for each plan
    lpostdict2 = {}
    for key2 in keywords2:
        # generate topic counts to populate dictionary frame indexed to topic
        positions2 = [i for i,w in enumerate(wordlist) if w.lower() == key2]
        lposdict2[key2] = positions2
    
    for key1, positions1 in lposdict1.items():
        for key2, positions2 in lposdict2.items():

            # create subtraction matrix for each keyword pair
            key1_key2 = abs(np.subtract(np.array(positions1), np.array(positions2)[:,None])) # with different array shapes
            # count occurences in each matrix where distance between keywords is <100 words + store in dictionary 
            proxim[key1, key2] = np.count_nonzero(key1_key2 < 100)
            # convert dictionary to dataframe series/column for each municipality
            prox = pd.DataFrame.from_dict(proxim, orient='index', columns=[plan.split(", ")[0]])
    
    plan_col.append(prox)
    
    # add each muni column generated to a single dataframe
    muniprox = pd.concat(plan_col, axis=1)

muniprox

,Alhambra
"(food, soil)",1
"(food, remediation)",1
"(food, sustainability)",0
"(food, climate)",6
"(food, environment)",2
"(food, safety)",12
"(food, justice)",13
"(food, public)",15
"(food, residential)",0
"(garden, soil)",1


In [ ]:
# reorienting table the keyword pair frequencies can joined to other gdfs
muniproxt = muniprox.transpose()
muniproxt

### (C) Modified Topic Modeling: How Are Municipalities Treating Food & Urban Agriculture as Policy Priority?

The last exercise focused on identifying/verifying whether food and urban agriculture was being talked about in relation to a specific set of topics and keywords (health, safety, sustainability, climate). This is a variation on topic modeling that focuses more agnostically on finding the common words/topics around mentions of food and urban agriculture in general plans to gauge how food is talked about in municipalities more generally. 

In [60]:
# creating a keyword subset dictionary for topic modeling purposes
textdictsub = {}

keyloc = ['food', 'agriculture', 'garden', 'farm', 'fruit', 'vegetable', 'animal']

for plan in genplan:
    for key in keyloc:
        # text positions
        textpositionsub = [i for i in range(len(plan)) if plan.startswith(key, i)] # adapted: https://www.geeksforgeeks.org/python-all-occurrences-of-substring-in-string/#
        textdictsub[key, plan.split(", ")[0]] = textpositionsub

textdictsub

{('food', 'Alhambra General Plan'): [28264,
  92775,
  102247,
  107926,
  109065,
  113179,
  178165,
  198174,
  198866,
  209796,
  209884,
  210207],
 ('agriculture', 'Alhambra General Plan'): [144819],
 ('garden', 'Alhambra General Plan'): [37125,
  39045,
  65339,
  97617,
  97662,
  97853,
  98611,
  102899,
  116871,
  118153,
  166246,
  166449,
  210144],
 ('farm', 'Alhambra General Plan'): [11630],
 ('fruit', 'Alhambra General Plan'): [101651],
 ('vegetable', 'Alhambra General Plan'): [98417, 101658, 210134],
 ('animal', 'Alhambra General Plan'): [124235],
 ('food', 'City of Commerce 2020 General Plan'): [8724, 98911, 575597, 621711],
 ('agriculture', 'City of Commerce 2020 General Plan'): [120021],
 ('garden', 'City of Commerce 2020 General Plan'): [],
 ('farm', 'City of Commerce 2020 General Plan'): [193528,
  318124,
  318336,
  318611],
 ('fruit', 'City of Commerce 2020 General Plan'): [],
 ('vegetable', 'City of Commerce 2020 General Plan'): [],
 ('animal', 'City of Com

In [132]:
# loop: try getting all segs for one keyword in Alhambra plan

# keyword
key = 'food'

# empty frame to store keys and created word segments for key
foodsegs = {}

# list: keyword positions for "food" in Alhambra
foodpositionsr = textdict['food']

# loop: for ea position of key in plan, create segment = +/- 200 words of key mention
for foodpr in foodpositionsr: 
    # add row to dict for ea unique key mention and corresponding word segments
    foodsegs[key, foodpr] = Alhambra[foodpr-800:foodpr+800]
    
    # turn dict into df: store ea segment for keyword mention
    foodsegsdf = pd.DataFrame.from_dict(foodsegs, orient='index', columns=['foodseg'])
    foodsegsdf.index.name = 'key'
    
    # add muni ID column
    muniID = Alhambra.lower().find("city of ")
    foodsegsdf['municipality'] = Alhambra.split(", ")[0]
    
foodsegsdf

,foodseg,municipality
key,,
"(food, 28264)",ications system accessible throughout the comm...,Alhambra General Plan
"(food, 92775)",y Crafting land use policies that allow for fi...,Alhambra General Plan
"(food, 102247)",ultural amenities Alhambra hosts a number of c...,Alhambra General Plan
"(food, 107926)",ate environmental justice policies into their ...,Alhambra General Plan
"(food, 109065)",o CalEnviroscreen a program associated with th...,Alhambra General Plan
"(food, 113179)",cs Policy Numbers Land Use and Community Desig...,Alhambra General Plan
"(food, 178165)",resources available to respond when a public h...,Alhambra General Plan
"(food, 198174)",he community and input received from the publi...,Alhambra General Plan
"(food, 198866)",the communitys quality of life and economic v...,Alhambra General Plan


In [134]:
# getting all segs for all keywords in Alhambra plan

# empty frame to store keys and created word segments for key
segs = {}

# for ea key in keylist, for ea position of key in plan, create segment = +/- 200 words of key mention
# using ea position for ea unique mention of key:
for keyword, positions in textdict.items():
    for position in positions:
        # add row to dict for ea unique key mention and corresponding word segments
        segs[keyword, position] = Alhambra[position-800:position+800]
            
    # turn dict into df: store ea segment for ea keyword in keyloc (for ea plan in genplan)
    segsdf = pd.DataFrame.from_dict(segs, orient='index', columns=['seg'])
    segsdf.index.name = 'key'
   
    # add muni ID column
    muniID = Alhambra.lower().find("city of ")
    segsdf['municipality'] = Alhambra.split(", ")[0]

segsdf

,seg,municipality
key,,
"(food, 28264)",ications system accessible throughout the comm...,Alhambra General Plan
"(food, 92775)",y Crafting land use policies that allow for fi...,Alhambra General Plan
"(food, 102247)",ultural amenities Alhambra hosts a number of c...,Alhambra General Plan
"(food, 107926)",ate environmental justice policies into their ...,Alhambra General Plan
"(food, 109065)",o CalEnviroscreen a program associated with th...,Alhambra General Plan
...,...,...
"(justice, 108710)",blic facilities food access safe and sanitary ...,Alhambra General Plan
"(justice, 109345)",nmental justice communities can be defined bot...,Alhambra General Plan
"(justice, 110740)",igure Disadvantaged Communities ALHAMBRA Gener...,Alhambra General Plan


In [66]:
# getting all segs for all keywords in all plans

# empty frame to store keys and created word segments for key
allsegs = {}

# for ea key in keylist, for ea position of key in plan, create segment = +/- 200 words of key mention
# using ea position for ea unique mention of key:
for plan in genplan:
    for keyword, positions in textdictsub.items():
        for position in positions:
            # add row to dict for ea unique key mention and corresponding word segments
            allsegs[keyword, position] = plan[position-800:position+800]
            
        # turn dict into df: store ea segment for ea keyword in keyloc (for ea plan in genplan)
        allsegsdf = pd.DataFrame.from_dict(allsegs, orient='index', columns=['seg'])
        allsegsdf.index.name = 'key'
    
allsegsdf

,seg
key,
"((food, Alhambra General Plan), 28264)",rial and residential uses Vernons city boundar...
"((food, Alhambra General Plan), 92775)",that the City owns of these units original dat...
"((food, Alhambra General Plan), 102247)",between and percent of the Los Angeles County ...
"((food, Alhambra General Plan), 107926)",tardation but shall not include other handicap...
"((food, Alhambra General Plan), 109065)",ersons with special needs and disabilities Lar...
...,...
"((animal, Vernon_General_Plan), 164757)",s located near the intersection of nd Street a...
"((animal, Vernon_General_Plan), 290685)",broad homogeneous area Glossary Vernon General...
"((animal, Vernon_General_Plan), 290784)",The Regional Housing Needs Assessment RHNA is...


In [67]:
# converting the column containing the segments in the dataframe series (seg) into a list of strings
allsegslist = allsegsdf.seg.values

# turning list of segment strings into a list of segment lists
# list of strings: ea string is a narrow word segment surrounding each unique keyword mention
planseglists = [[word for word in word_tokenize(seg.lower())
                 if word not in swords and len(word)>2] for seg in allsegslist]

In [68]:
print('There are {} total segments in planseglists.'.format(len(planseglists)))

There are 66 total segments in planseglists.


Taking the list of segments for all unique keyword mentions for each keyword for each plan and passing it through GENSIM for topic modeling.

In [ ]:
# generating topic model: start w/ 5 topics
dictionary = gensim.corpora.Dictionary(planseglists)
corpus = [dictionary.doc2bow(planseg) for planseg in planseglists]
# LdaMulticore uses multiple cores (thus, it runs faster); if you have problems, try replacing LdaMulticore with LdaModel
model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=5)

# show topics
model.show_topics()

### (D) Environmental Justice at the Municipal Level: Are Food Systems Part of the Picture?

This last exercise is focused on replicating the previous exercises exclusively on the Environmental Justice Elements of General Plans (for those plans that contain them). In 2016, the state of California passed legislation requiring all municipalities to include environmental justice planning in their overall city planning.